# Homework 3

#### EE-556 Mathematics of Data - Fall 2022

In this homework we will solve constrained optimization problems.
If you worked with somebody else on this homework, fill in their names here:


#### Setup

<div class="alert alert-danger">
    <b>Warning</b> You will need a couple of libraries for this homework, which you can install with pip by executing the block below.
</div>

In [ ]:
!pip install -r requirements.txt

In [ ]:
from lib.part1.deblur_lib import *

#### Introduction

We have seen in the lectures that given an optimization task whose iterates live on a  convex subset $\mathcal{X} \subset \mathbb{R}^n$ we can ensure the iterates of our algorithms stay within $\mathcal{X}$ in one of two ways. 

In the first way, we use projections, computed via the proximal operator $\text{prox}_{\delta_\mathcal{X}}$. In the second way, we use linear minimization oracles  $\text{lmo}_\mathcal{X}$ within a conditional gradient framework
that take simplicial combinations of elements from the set X whereby producing iterates remaining in $\mathcal{X}$.

The following exercises will help you understand what kind of computations are involved for each of these two operators, and how their computational complexity compares. For this we will work with $\mathcal{X}$ being the set of low-rank matrices defined via the nuclear norm ball $\mathcal{X}=\lbrace X:X\in \mathbb{R}^{p\times n}, \Vert X  \Vert_* \leq \kappa \rbrace$ with $\kappa$ being the radius of the zero-centered nuclear norm ball.

We will first mathematically study properties of  the proximal operator $\text{prox}_{\delta_\mathcal{X}}$  (**1.1** ) and the linear minimization oracles $\text{lmo}_\mathcal{X}$ (**1.2**) onto this $\mathcal{X}$ and then compare them empirically by implementing movie recommender system (**1.3**) and image deblurring (**1.4**) algorithms.




## 1.1 Computing projections onto $\mathcal{X}$


#### Question 1.1.1 (2 pts)

Recall that given a set $\mathcal{X} \subset \mathbb{R}^{p \times m}$, its corresponding
    projection operator is given by
    $\mathrm{proj}_{\mathcal{X}} (\boldsymbol{Z}) = \mathop{\mathrm{arg\,min}}\limits_{{\bf X}\in \mathcal{X}}\{ \|{\bf X}- \boldsymbol{Z}\|_F^2\}, \; \forall \boldsymbol{Z}\in  \mathbb{R}^{p \times m}$.
    Using the definition of the proximal operator given in class, show
    the equivalence between the projection operator and the proximal
    operator:
    $$\mathrm{proj}_{\mathcal{X}} (\boldsymbol{Z})  = \mathrm{prox}_{\delta_{\mathcal{X}}}(\boldsymbol{Z}),$$
    where $\delta_{\mathcal{X}}$ is the indicator function of
    $\mathcal{X}$ i.e.  $\delta_{\mathcal{X}}({\bf Y}) = \begin{cases} 0, \text{ if } {\bf Y}\in \mathcal{X} \\ +\infty, \text{ o.w. } \end{cases}$

<font color="blue">

Fill your answer here.

#### Question 1.1.2 (3 pts)

The projection operator of convex sets has an interesting and useful property: it is non-expansive. Mathematically, we write:
$$\begin{align}
        \|\mathrm{proj}_{\mathcal{X}} (\textbf{x}) - \mathrm{proj}_{\mathcal{X}} (\textbf{y})\| \leq \|\textbf{x}- \textbf{y}\|, \forall \textbf{x}, \textbf{y}\in \mathbb{R}^p,\tag{1}\label{eq:non-expansivity}
\end{align}$$
where $\|\cdot\|$ denotes the usual Euclidean norm
and $\mathcal{X}$ is a non-empty, closed and convex set. For keeping
things simple, in this point we use the space of vectors
$\mathbb{R}^p$ in which $\mathcal{X} \in \mathbb{R}^p$ is a closed
convex set.

Informally, \eqref{eq:non-expansivity} means that the distance between the
*projections* of the two points onto $\mathcal{X}$ will be *no
greater* than the distance between the points themselves.
Conversely, for non-convex sets, this does not hold (you can try
building a counterexample for a doughnut-shaped set).

Prove inequality \eqref{eq:non-expansivity} starting from the equivalent
characterization of the Euclidean projection:
$\textbf{z}^* = \mathrm{proj}_{\mathcal{X}} (\textbf{x}) \iff \langle \textbf{x}- \textbf{z}^*, \textbf{z}- \textbf{z}^*\rangle \leq 0, \, \forall \textbf{z}\in \mathcal{X}$.

<font color="blue">

Fill your answer here.

#### Question 1.1.3.  (4 pts)

Let ${\boldsymbol Z}= {\boldsymbol U}\boldsymbol{\Sigma} {\boldsymbol V}^\top$
    be the singular value decomposition of
    ${\boldsymbol Z}\in \mathbb{R}^{p \times m}$. Denote the diagonal of
    $\boldsymbol{\Sigma} \in \mathbb{R}^{s \times s}$ by a vector
    $\sigma \in \mathbb{R}^{s}$, where $s = \min \{ p, m \}$. Let
    $\sigma^{\ell_1}$ be the projection of $\sigma$ onto the
    $\ell_1$-norm ball
    $\{ \textbf{x}: \textbf{x}\in \mathbb{R}^{s} , \left\Vert  \textbf{x} \right\Vert_1 \leq \kappa  \}$
    with radius $\kappa$. Show that the projection of this matrix onto
    the nuclear norm ball
    $\mathcal{X} = \{ {\boldsymbol X}: {\boldsymbol X}\in \mathbb{R}^{p \times m} , \| {\boldsymbol X}\|_\ast \leq \kappa \}$
    can be computed by projecting $\sigma$ onto the $\ell_1$ norm ball,
    i.e.,
    $$\mathrm{proj}_\mathcal{X} ({\boldsymbol Z}) = {\boldsymbol U}\boldsymbol\Sigma^{\ell_1} {\boldsymbol V}^\top,$$
    where $\Sigma^{\ell_1} \in \mathbb{R}^{s \times s}$ denotes the
    diagonal matrix with diagonal $\sigma^{\ell_1}$.

(Hint: Use Mirsky's inequality:
    $\| {\boldsymbol X}- {\boldsymbol Z}\|_F \geq \| \boldsymbol{\Sigma}_{{\boldsymbol X}} - \boldsymbol{\Sigma}_{{\boldsymbol Z}}\|_F$,
    where
    $\boldsymbol{\Sigma}_{{\boldsymbol X}}, \boldsymbol{\Sigma}_{{\boldsymbol Z}} \in \mathbb{R}^{s \times s}$
    are the diagonal matrices of the singular values of
    ${\boldsymbol X}, {\boldsymbol Z}$ respectively.)

<font color="blue">

Fill your answer here.

## 1.2 Computing the linear minimization oracle of $\mathcal{X}$

#### Question 1.2.1 (4 pts)

Problem 1.1 shows that projection onto the nuclear norm ball requires
computing the singular value decomposition. The computational complexity
of the singular value decomposition is $\mathcal{O}(\min(m^2p,mp^2))$,
which can easily become a computational bottleneck if $m$ or $p$ are
large. This bottleneck increased the popularity of algorithms that
leverage the linear minimization oracle (lmo) instead (e.g.,
[Jaggi2013](https://proceedings.mlr.press/v28/jaggi13.html), [yurtsever2018](http://proceedings.mlr.press/v80/yurtsever18a)):
$$\text{lmo}_{\mathcal{X}}({\boldsymbol Z})  = \arg \min_{{\boldsymbol X}\in \mathcal{X}} \langle {\boldsymbol X},{\boldsymbol Z}\rangle \qquad \text{where}\qquad \langle {\boldsymbol X},{\boldsymbol Z}\rangle = \text{Tr}({\boldsymbol Z}^\top{\boldsymbol X}).$$
Note that $\text{lmo}_\mathcal{X}({\boldsymbol Z})$ is not single valued
in general. With abuse of terminology, when we say that we compute the
lmo, we actually mean that we compute an instance ${\boldsymbol X}$ such
that ${\boldsymbol X}\in \text{lmo}_\mathcal{X}({\boldsymbol Z})$.

Show that the lmo$_\mathcal{X}$ when $\mathcal{X}$ is the nuclear norm
ball:
$\mathcal{X} = \{ {\boldsymbol X}: {\boldsymbol X}\in \mathbb{R}^{p \times m} , \| {\boldsymbol X}\|_\ast \leq \kappa \}$
gives the following output:
$$-\kappa ~ \! \mathbf{u}\mathbf{v}^T   \in  \text{lmo}_{{\mathcal{X}}}({\boldsymbol Z}) ,$$
where $\mathbf{u}$ and $\mathbf{v}$ are the left and right singular
vectors that correspond to the largest singular value of
${\boldsymbol Z}$.

(Hint: By definition
$\kappa ~ \! \mathbf{u}\mathbf{v}^T \in \mathcal{X}$. You just need to
show
$\langle {\boldsymbol X},{\boldsymbol Z}\rangle \geq \langle -\kappa ~ \! \mathbf{u}\mathbf{v}^T,{\boldsymbol Z}\rangle$
for all ${\boldsymbol X}\in \mathcal{X}$.)

<font color="blue">

Fill your answer here.

## 1.3 Comparing the scalability of $\mathrm{proj}_{\mathcal{X}} (\boldsymbol{Z})$ and $\mathrm{lmo}_{\mathcal{X}} (\boldsymbol{Z})$

In this exercise, we will compare the execution time of
$\mathrm{proj}_{\mathcal{X}} (\boldsymbol{Z})$ and
$\mathrm{lmo}_{\mathcal{X}} (\boldsymbol{Z})$ on two datasets provided
to you in the codes. These datasets consist of the ratings given by
MovieLens users to movies in a given list. The 100k dataset consists of
100,000 ratings from 1000 users on 1700 movies. The 1M dataset consists
of 1 million ratings from 6000 users on 4000 movies.

As you likely figured out already from the numbers above, users do not
rate all of the movies, and therefore, we model the ratings as entries
of a low-rank matrix, where rows correspond to different users and
columns correspond to different movies. A classical task in machine
learning is to predict the value of the missing entries, which is called
the matrix completion problem.

Many other tasks can be formulated as convex minimization problems,
constrained to the nuclear-norm ball, which captures a low rank model
since it is the atomic norm of rank-1 matrices (see Lecture 4). A good
optimization algorithm must ensure feasibility in a scalable way: For
instance, the famous Netflix competition data consists of 100480507
ratings that 480189 users gave to 17770 movies (much bigger than the
datasets above). Projecting a matrix of this size onto the nuclear-norm
ball is indeed demanding.

#### Question 1.3.1 (4 pts)

Implement the projection operator as a function called `proj_nuc` below. You can use the helper function `proj_L1` we define here from the `projL1.py` file.

In [ ]:
from lib.part1.projL1 import projL1

Set $\kappa = 5000$ and measure the computation time of the
    projection operator with the 100k and the 1M MovieLens dataset using our provided helper code, which loads the datasets, constructs the data matrix, and times the evaluation of
    the projection operator. Write the values you get in a markdown cell.
    Run and report the average timing.

In [ ]:
def proj_nuc(Z, kappa):
    """ This function implements the projection onto the nuclear norm ball.
    """
  
    proj = ???
    
    return proj

In [ ]:
eval_completion("100k_MovieLens", proj_nuc, kappa=5000)

In [ ]:
# NOTE: This one can take few minutes!
eval_completion("1M_MovieLens", proj_nuc, kappa=5000)

<font color="blue">

Fill your answer here.

#### Question 1.3.2 (4 pts)

Implement the lmo with ${\mathcal{X}}$ as a function called
    `lmo_nuc`  below. Set again $\kappa = 5000$ and measure the
    computation time for the 100k and 1M Movielens datasets. Compare these values with
    the computation time of the projection operator. You are allowed to use `scipy` (the library is already loaded).

In [ ]:
def lmo_nuc(Z, kappa):
    """ This function implements the lmo operator for the nuclear norm ball constraint.
    """
    
    Zhat = ???
    
    return Zhat

In [ ]:
eval_completion("100k_MovieLens",lmo_nuc,kappa=5000)

In [ ]:
eval_completion("1M_MovieLens",lmo_nuc,kappa=5000)

<font color="blue">

Fill your answer here.

## 1.4 Frank-Wolfe for blind image deblurring

You are working with the local police to help identify a license plate
of a car involved in a crime scene investigation. Unfortunately, the
CCTV image of the car is blurry. In this exercise, we simulate this
scenario with a deblurred license plate image found from the
internet.

Deblurring is an instance of the blind deconvolution problem: Given two
unknown vectors $\textbf{x},  {\boldsymbol w}\in \mathbb{R}^L$, we
observe their circular convolution
$\textbf{y}=  {\boldsymbol w}*\textbf{x}$, i.e.,
$$y_\ell = \sum_{\ell'=1}^L w_{\ell'} x_{\ell - \ell' + 1},$$ where the
index $\ell - \ell' + 1$ in the sum is understood to be modulo $L$.

Blind deconvolution seeks to separate ${\boldsymbol w}$ and
$\textbf{x}$, given $\textbf{y}$. The operative word *blind* comes from
the fact that we do not have much prior information about the signals.
In this case, what we can assume is that ${\boldsymbol w}$ and
$\textbf{x}$ belong to *known* subspaces of $\mathbb{R}^L$ of dimension
$K$ and $N$, i.e., we write $$\begin{aligned}
{\boldsymbol w}&= {\boldsymbol B}{\boldsymbol h}, \quad {\boldsymbol h}\in \mathbb{R}^K \\
\textbf{x}&= {\boldsymbol C}{\boldsymbol m}, \quad {\boldsymbol m}\in \mathbb{R}^N
\end{aligned}$$ for some $L \times K$ matrix ${\boldsymbol B}$ and
$L \times N$ matrix ${\boldsymbol C}$. The columns of these matrices
form bases for the subspaces in which ${\boldsymbol w}$ and $\textbf{x}$
live.

As we have seen in Homework 1, natural images have sparse wavelet
expansions. Hence, the image $\textbf{x}$ can be expressed as
$\textbf{x}= {\boldsymbol C}{\boldsymbol m}$ with ${\boldsymbol C}$ is
the matrix formed by a subset of the columns of the wavelet transform
matrix. In addition, the blur kernel ${\boldsymbol w}$ is typically due
to simple or "sparse" motion, which can be written as
${\boldsymbol w}= {\boldsymbol B}{\boldsymbol h}$ with ${\boldsymbol B}$
is the matrix formed by a subset of the columns of the identity matrix.

In deblurring, $\textbf{x}$ corresponds to the image we want to recover
(i.e., the license plate) and ${\boldsymbol w}$ to a 2D blur kernel.
Thus, the 2D convolution $\textbf{y}=  {\boldsymbol w}*\textbf{x}$
produces a blurred image. We assume that we know or can estimate the
support of the blur kernel (i.e., the location of its nonzero elements).
In real applications, the support can be estimated by an expert using
the physical information such as the distance of object to the focus and
the camera, the speed of the camera and/or the object, camera shutter
speed, etc.

In this experiment, we use a very rough estimate for the support - a box
at the center of the domain, whose size we have roughly tuned.
Interestingly, it is possible to make the plate readable even in this
setting.

In [ ]:
from lib.part1.deblur_lib import setup_show
setup_show()

###  Reformulating the problem

We now reformulate the blind image deconvolution problem, so that we can
apply the constrained optimization algorithms we have seen in the
course. Let ${\boldsymbol b}$ be the $L$-point normalized discrete
Fourier transform (DFT) of the observation $\textbf{y}$, i.e,
${\boldsymbol b}= \mathbf{F} \textbf{y}$, where $F$ is the DFT matrix.
Then, ${\boldsymbol b}$ can be written as
${\boldsymbol b}= {\boldsymbol A}({\bf X})$ where
${\bf X}= {\boldsymbol h}{\boldsymbol m}^\top$ and ${\boldsymbol A}$ is
a linear operator. Explicit expression of this linear operator
${\boldsymbol A}$ is out of the scope of this homework, c.f.,
[Ahmed2014](https://ieeexplore.ieee.org/document/6680763/) for further details. This reformulation allows us to
express $\textbf{y}$, which is a nonlinear combination of the
coefficients of ${\boldsymbol h}$ and ${\boldsymbol m}$, as a linear
combination of the entries of their outer product
${\bf X}= {\boldsymbol h}{\boldsymbol m}^\top$. Note that given
${\boldsymbol B}$ and ${\boldsymbol C}$, recovering ${\boldsymbol m}$
and ${\boldsymbol h}$ from ${\boldsymbol b}$ is the same as recovering
$\textbf{x}$ and ${\boldsymbol w}$ from $\textbf{y}$.

Since ${\bf X}$ is a rank one matrix, we can use the nuclear norm to
enforce approximately low-rank solutions. Then, we can formulate the
blind deconvolution problem as follows:
$${\boldsymbol X}^\star \in \arg \min_{ {\boldsymbol X}} \bigg\{ \frac{1}{2} \| \mathbf{A}({\boldsymbol X}) - {\boldsymbol b}\|_2^2 :  \| {\boldsymbol X}\|_\ast \leq \kappa, ~{\boldsymbol X}\in \mathbb{R}^{p\times m}   \bigg\}, \tag{4}\label{eq:FWform}$$
where $\kappa > 0$ is a tuning parameter.

Note that our problem is constrained to the nuclear norm ball
$\mathcal{X} = \{ {\boldsymbol X}: {\boldsymbol X}\in \mathbb{R}^{p \times m} , \| {\boldsymbol X}\|_\ast \leq \kappa \}$.

We will apply the Frank-Wolfe algorithm to solve the optimization
problem given in \eqref{eq:FWform}. The Frank-Wolfe algorithm is one of the earliest
algorithms that avoids projections. Instead of projections, it leverages
lmos (for a very good survey see [Jaggi2013](https://proceedings.mlr.press/v28/jaggi13.html)):
$$\mathrm{lmo}(\nabla f ({\boldsymbol Z})) = \arg \min_{{\boldsymbol X}\in \mathcal{X}} ~ \langle \nabla f ({\boldsymbol Z}), {\boldsymbol X}\rangle,$$
where
$\mathcal{X} = \{ {\boldsymbol X}: \| {\boldsymbol X}\|_\ast \leq \kappa, ~{\boldsymbol X}\in \mathbb{R}^{p\times m} \}$
as in Part 1. It applies to the generic constrained minimization
template with a smooth objective function,
$\min_{\boldsymbol X}\{ f({\boldsymbol X}) : {\boldsymbol X}\in \mathcal{X}, \, \mathcal{X} \text{ - convex, compact}  \}$
as follows:

----


### Frank-Wolfe's algorithm

1. Choose ${\boldsymbol X}^0 \in\mathcal{X}$.

2. For $k=0, 1, \ldots$ perform:
  $$\begin{cases}
  \hat{{\bf X}}^k &:= \mathrm{lmo}(\nabla f ({\boldsymbol X}^k)), \\
  {\bf X}^{k+1} &:= (1-\gamma_k){\bf X}^k + \gamma_k\hat{{\bf X}}^k,
  \end{cases}$$ where $\gamma_k := {2}/{(k+2)}$.

---

#### Question 1.4.1 (4 pts)

Recall that the Frank-Wolfe algorithm applies only for
    smooth objectives. Show that the objective function is smooth in the sense its gradient is Lipschitz continuous.

<font color="blue">
    
Fill your answer here.

### Implementation of Frank-Wolfe

Complete the missing lines of the Frank-Wolfe update below.
We provide you the linear operators that you need to compute the LMO in the code. Note that we do not need to
    store and use the linear operator ${\boldsymbol A}$ in the ambient
    dimensions. In fact, for storage and arithmetic efficiency, we
    should avoid explicitly writing ${\boldsymbol A}$. You can find more
    details about this aspect as comments in the code.

In [ ]:
from lib.part1.deblur_lib import *

We will track three states:

- `X`: corresponding to $\boldsymbol X$
- `AX`: corresponding to $\boldsymbol{AX}$
- `k`: the iterate count

In [ ]:
def init_state():
    return OptState(X=0.0, AX=0.0, k=1)

#### Question 1.4.2 (2 points) Complete the gradient computation of $f(\boldsymbol X)$
You have access to the following operators and variables:
    
- `A_T(Z)`: Computes $\boldsymbol A^\top \boldsymbol Z$ for some $\boldsymbol Z$.
- `Aoper`: Computes $\boldsymbol A \boldsymbol X$ for 1-rank matrix $\boldsymbol X$ using its SVD decomposition.
- `b`: corresponds to $\boldsymbol b$

**Remark**: The `grad` method will receive `AX` and not `X` for computational efficiency reasons.

In [ ]:
f = Function(
    f = lambda AX: 0.5*np.linalg.norm(AX - b, 2)**2,
    grad = lambda AX: ???,
)

#### Question 1.4.3 (5 points) Complete the LMO

You are allowed to use `scipy`.

In [ ]:
def lmo(Grad, kappa=1000):
    """ This function implements the lmo operator.
    """

    Xhat = ???

    # Apply A to the rank 1 update
    AXhat = Aoper(topLe_vec, -kappa, topRe_vec.T)

    return (Xhat, AXhat)

#### Question 1.4.4 (5 points) Complete the Frank-Wolfe update

Fill in the missing update of `X` using the LMO from the previous exercise.

In [ ]:
def state_update(f, state):
    X, AX, k = state

    Xhat, AXhat = ???

    # Step size
    gamma_k = ???

    # Update X
    next_X = ???

    # Update A*X
    next_AX = (1.0-gamma_k)*AX + gamma_k*(AXhat)

    return OptState(next_X, next_AX, k+1)
    

In [ ]:
opt_algorithm = OptAlgorithm("FrankWolfe", init_state, state_update)

#### Question 1.4.5 (3 points) Run Frank-Wolfe

Tune the 
    parameter $\kappa$ until the license plate number becomes readable.
    What is the license plate number? 


In [ ]:
# Run Frank-Wolfe's method
xFW = run_frank_wolfe(f, opt_algorithm)

<font color="blue">
    
Fill your answer here.
